In [2]:
import numpy as np

## Markov Decision Process



Markov Decision Process (MDP) represents environments with state-spaces that allow an agent to make decisions. In search algorithms, we return a sequence of actions that lead to the goal state, and actions always translate to states. In reinforcement learning, we return a sequence of states that lead to the goal state.  To do this, the agent is rewarded for taking actions that lead to the goal state. MDP is a richer environment representation that allows randomness that other machine learning strategies cannot handle. MDP uses stochastic transition probabilities to model the environment. Stochastic transitions include randomness, for example, the agent might move in a direction and a condition might occur which puts the agent in an unmodeled condition. In robotics, the ability to maintain function in uncertainties is called resiliance. Therefore, MDP is an important tool for kinematic resiliance.

Probability distributions include $s'$ what state we end up in, $s$ given the current state that we are in, and $a$ the action that we choose.

$$Pr(s|s', a)=0.2$$

This representation means three things: (i) the next state depends only one the previous state, (ii)  the probability function is the description and representation of the environment, (iii) the agent generates its own data.

To determine how to acquire 'good' rewards, we propose a policy. A policy, represented with $\pi$, is a function of states that maps states to actions, and tells which action to take in each state.

A policy is different than a sequence of actions. In sequence of actions, if the transition takes the agent to an unmodeled or uncertain state, the algorithm runs out of options. However, policy can be evaluated in the uncertain state.

We consider a rectangular gridworld representation (see below) of a simple finite Markov Decision Process (MDP). The cells of the grid correspond to the states of the environment. At each cell, four actions are possible: north, south, east, and west, which deterministically cause the agent to move one cell in the respective direction on the grid. Actions that would take the agent off the grid leave its location unchanged, but also result in a reward of -1. In other words, rewards are positive for goals, negative for running into undesirable states, and zero the rest of the time.

$$
\begin{array}{|c|c|c|c|}
\hline
0 & 0 & 0 & 0 \\
\hline
0 & 0 & -1 & 0 \\
\hline
\colorbox{blue}{}& 0 & -1 & 1 \\
\hline
\end{array}
$$



Next we define the rewards for the states. Those will be of +1 for the state that is desirable, of -1 for states that have to be avoided and of 0 for all other states. The Bellman equation expresses a relationship between the value of a state and the values of its successor states and must hold for each state for the value function $v_{\pi}$

$$ v_{\pi}(s) = \sum_a \pi(a \mid s) \sum_{s'| r} p(s', r \mid s, a) [ r + \gamma v_{\pi}(s')] $$

where the actions, $a$ are taken from the set $A(s)$ that the next states, $s'$ are taken from the set $S$ and that the rewards, $r$ are taken from the set $R$.

Suppose the agent selects all four actions with equal probability in all states. The figure below shows the value function, $v_{\pi}$, for this policy, for the discounted reward case with $\gamma=0.9$

$$
\begin{array}{|c|c|c|c|}
\hline
0.42 & 0.48 & 0.52 & 0.63 \\
\hline
0.38 & 0.43 & \color{orange}{-1} & 0.72 \\
\hline
0.34 & 0.38 & \color{orange}{-1} & \color{green}{+1} \\
\hline
\end{array}
$$

Therefore the agent movement can be tracked so:

$$
\begin{array}{|c|c|c|c|}
\hline
 & \textcolor{lightblue}{\rightarrow} & \textcolor{lightblue}{\rightarrow} & \textcolor{lightblue}{\downarrow} \\
\hline
\textcolor{lightblue}{\rightarrow} & \textcolor{lightblue}{\uparrow} & \textcolor{orange}{-1} & \textcolor{lightblue}{\downarrow} \\
\hline
\textcolor{lightblue}{\uparrow\rightarrow} & \textcolor{lightblue}{\uparrow} & \textcolor{orange}{-1} & \textcolor{green}{+1} \\
\hline
\end{array}
$$


The logic can be seen in the code snippet below

In [5]:
#Hyperparameters
SMALL_ENOUGH = 0.005
GAMMA = 0.9
NOISE = 0.1

#Define all states
all_states=[]
for i in range(3):
    for j in range(4):
            all_states.append((i,j))

#Define rewards for all states
rewards = {}
for i in all_states:
    if i == (1,2):
        rewards[i] = -1
    elif i == (2,2):
        rewards[i] = -1
    elif i == (2,3):
        rewards[i] = 1
    else:
        rewards[i] = 0

#Dictionary of possible actions. We have two "end" states (1,2 and 2,2)
actions = {
    (0,0):('D', 'R'),
    (0,1):('D', 'R', 'L'),
    (0,2):('D', 'L', 'R'),
    (0,3):('D', 'L'),
    (1,0):('D', 'U', 'R'),
    (1,1):('D', 'R', 'L', 'U'),
    (1,3):('D', 'L', 'U'),
    (2,0):('U', 'R'),
    (2,1):('U', 'L', 'R'),
    }

#Define an initial policy
policy={}
for s in actions.keys():
    policy[s] = np.random.choice(actions[s])

#Define initial value function
V={}
for s in all_states:
    if s in actions.keys():
        V[s] = 0
    if s ==(2,2):
        V[s]=-1
    if s == (1,2):
        V[s]=-1
    if s == (2,3):
        V[s]=1

iteration = 0
while True:
    biggest_change = 0
    for s in all_states:
        if s in policy:

            old_v = V[s]
            new_v = 0

            for a in actions[s]:
                if a == 'U':
                    nxt = [s[0]-1, s[1]]
                if a == 'D':
                    nxt = [s[0]+1, s[1]]
                if a == 'L':
                    nxt = [s[0], s[1]-1]
                if a == 'R':
                    nxt = [s[0], s[1]+1]

                #Choose a new random action to do (transition probability)
                random_1=np.random.choice([i for i in actions[s] if i != a])
                if random_1 == 'U':
                    act = [s[0]-1, s[1]]
                if random_1 == 'D':
                    act = [s[0]+1, s[1]]
                if random_1 == 'L':
                    act = [s[0], s[1]-1]
                if random_1 == 'R':
                    act = [s[0], s[1]+1]

                #Calculate the value
                nxt = tuple(nxt)
                act = tuple(act)
                v = rewards[s] + (GAMMA * ((1-NOISE)* V[nxt] + (NOISE * V[act])))
                if v > new_v: #Is this the best action so far? If so, keep it
                    new_v = v
                    policy[s] = a

       #Save the best of all actions for the state
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

   #See if the loop should stop now
    if biggest_change < SMALL_ENOUGH:
        break
    iteration += 1

Value iteration algorithm optimizes policies to navigate statespace.


Future Directions in Reinforcement Learning
- Extrapolating the learning to uncertain states
- Sample efficiency
- Multi-agent RL evolution
- Robustness